In [1]:
!python -V

Python 3.9.12


In [6]:
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Import MLFlow and declare your ML Experiment

In [25]:
import mlflow

## Set tracking uri
this is where all info regarding each run in the experiment will be stored

In [26]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

## Experiment name

In [27]:
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

# MLFlow UI

Browser based UI which shows our ML experiments with lot of different logged things such as `runs`, `metrics`, `loss`, `models`, etc.  
More on the same can be find [here](https://www.mlflow.org/docs/latest/tracking.html#concepts)

To access MLFlow UI, 
Run the following command in your working directory 
```bash
mlflow ui --backend-store-uri sqlite:///mlflow.db
```

![startup command](./images/startup_mlflow.jpg)


Open up http://127.0.0.1:5000/ in your browser and you should be able to see our experiment :  
![ui](./images/ui_mlflow.jpg)

# Data Preparation

In [12]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [14]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [15]:
len(df_train), len(df_val)

(73908, 61921)

In [16]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [17]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# Usual flow.... 🥱

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.4795387205188755

In [22]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

# MLFlow 🚀

## How to track ?

Everything inside :

```python
with mlflow.start_run()
```

will be tracked if logged properly

## How to log ?

* `mlflow.log_param()` logs a single key-value param in the currently active run. The key and value are both strings. Use `mlflow.log_params()` to log multiple params at once.

* `mlflow.log_metric()` logs a single key-value metric. The value must always be a number. MLflow remembers the history of values for each metric. Use mlflow.log_metrics() to log multiple metrics at once.

* `mlflow.set_tag()` sets a single key-value tag in the currently active run. The key and value are both strings. Use mlflow.set_tags() to set multiple tags at once.

* m`lflow.log_artifact()` logs a local file or directory as an artifact, optionally taking an artifact_path to place it in within the run’s artifact URI. Run artifacts can be organized into directories, so you can place the artifact in a directory this way.

* `mlflow.log_artifacts()` logs all the files in a given directory as artifacts, again taking an optional artifact_path.

* `mlflow.get_artifact_uri()` returns the URI that artifacts from the current run should be logged to.

Source : https://www.mlflow.org/docs/latest/tracking.html#logging-functions

In [31]:
with mlflow.start_run():
    
    # set tags to later filter out runs w.r.t those tags
    # e.g. later we can filter out runs made by developer named Pathik
    mlflow.set_tag("developer", "Pathik")
    
    # to track which data was used a train and test for this run 
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    # track artifact which is locally stored at location_path
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

## So wut ?🤔

If you checked the MLFlow UI after running above cell,  
You should be able to see our previous `run` : 
![overview_output_run_1](./images/overview_output_run_1.jpg)

Clicking on the time will give you a in-depth output  
![output_run_1](./images/output_run_1.jpg)

## Compare runs
Lets say we ran the above run again with a different `alpha=0.5`

In MLFLow UI, you will be able to see new run :  
![overview_output_run_2](./images/overview_output_run_2.jpg)

Let's say you wanna compare w.r.t `alpha` which run is giving best `rmse`  
then check out the tickboxes present on left of each run and select `Compare`

![compare_run](./images/run_comparison.jpg)

if you scroll down more you'll get to see more comparisons  
e.g. `rmse` difference in both runs

![rmse_compare](./images/rmse_compare.jpg)

since we just use two runs things aren't that impressive 

Lets make things a bit complex...

![complex meme gif](https://c.tenor.com/hh6n7Ou_OnUAAAAC/math-hangover.gif)

In [30]:
import xgboost as xgb

In [38]:
# for finding hyperparameters which will minize our objective 

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

# check it out here : http://hyperopt.github.io/hyperopt/

In [34]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [35]:
def objective(params):
    with mlflow.start_run(): # to track each run whenever this gets called later while optimizing 
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [36]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [37]:
# minimize our objective
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

# this will take some time so have a coffe ☕
# please scroll down for next part as output is just toooo long ... :(

[15:48:10] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.  
[0]	validation-rmse:16.05726                                                                                            
[1]	validation-rmse:12.61598                                                                                            
[2]	validation-rmse:10.37673                                                                                            
[3]	validation-rmse:8.96403                                                                                             
[4]	validation-rmse:8.09671                                                                                             
[5]	validation-rmse:7.57505                                                                                             
[6]	validation-rmse:7.25689                                                                                             
[7]	validation-rmse:7.06266     

[133]	validation-rmse:6.66667                                                                                           
[134]	validation-rmse:6.66662                                                                                           
[135]	validation-rmse:6.66670                                                                                           
[136]	validation-rmse:6.66654                                                                                           
[137]	validation-rmse:6.66644                                                                                           
[138]	validation-rmse:6.66614                                                                                           
[139]	validation-rmse:6.66605                                                                                           
[140]	validation-rmse:6.66620                                                                                           
[141]	validation-rmse:6.66612   

[267]	validation-rmse:6.66164                                                                                           
[268]	validation-rmse:6.66173                                                                                           
[269]	validation-rmse:6.66182                                                                                           
[270]	validation-rmse:6.66200                                                                                           
[271]	validation-rmse:6.66181                                                                                           
[272]	validation-rmse:6.66184                                                                                           
[273]	validation-rmse:6.66191                                                                                           
[274]	validation-rmse:6.66181                                                                                           
[275]	validation-rmse:6.66174   

[1]	validation-rmse:16.21247                                                                                            
[2]	validation-rmse:14.34714                                                                                            
[3]	validation-rmse:12.81333                                                                                            
[4]	validation-rmse:11.56841                                                                                            
[5]	validation-rmse:10.55704                                                                                            
[6]	validation-rmse:9.74532                                                                                             
[7]	validation-rmse:9.09518                                                                                             
[8]	validation-rmse:8.58001                                                                                             
[9]	validation-rmse:8.17400     

[135]	validation-rmse:6.50993                                                                                           
[136]	validation-rmse:6.50981                                                                                           
[137]	validation-rmse:6.50945                                                                                           
[138]	validation-rmse:6.50937                                                                                           
[139]	validation-rmse:6.50893                                                                                           
[140]	validation-rmse:6.50822                                                                                           
[141]	validation-rmse:6.50792                                                                                           
[142]	validation-rmse:6.50754                                                                                           
[143]	validation-rmse:6.50729   

...

[400]	validation-rmse:6.42590                                                                                           
[401]	validation-rmse:6.42579                                                                                           
[402]	validation-rmse:6.42559                                                                                           
[403]	validation-rmse:6.42521                                                                                           
[404]	validation-rmse:6.42484                                                                                           
[405]	validation-rmse:6.42455                                                                                           
[406]	validation-rmse:6.42450                                                                                           
[407]	validation-rmse:6.42440                                                                                           
[408]	validation-rmse:6.42440   

[534]	validation-rmse:6.39464                                                                                           
[535]	validation-rmse:6.39442                                                                                           
[536]	validation-rmse:6.39422                                                                                           
[537]	validation-rmse:6.39399                                                                                           
[538]	validation-rmse:6.39368                                                                                           
[539]	validation-rmse:6.39318                                                                                           
[540]	validation-rmse:6.39265                                                                                           
[541]	validation-rmse:6.39221                                                                                           
[542]	validation-rmse:6.39205   

[668]	validation-rmse:6.37141                                                                                           
[669]	validation-rmse:6.37118                                                                                           
[670]	validation-rmse:6.37111                                                                                           
[671]	validation-rmse:6.37072                                                                                           
[672]	validation-rmse:6.37066                                                                                           
[673]	validation-rmse:6.37053                                                                                           
[674]	validation-rmse:6.37038                                                                                           
[675]	validation-rmse:6.37048                                                                                           
[676]	validation-rmse:6.37050   

[802]	validation-rmse:6.35435                                                                                           
[803]	validation-rmse:6.35435                                                                                           
[804]	validation-rmse:6.35430                                                                                           
[805]	validation-rmse:6.35430                                                                                           
[806]	validation-rmse:6.35420                                                                                           
[807]	validation-rmse:6.35357                                                                                           
[808]	validation-rmse:6.35346                                                                                           
[809]	validation-rmse:6.35334                                                                                           
[810]	validation-rmse:6.35321   

[936]	validation-rmse:6.34127                                                                                           
[937]	validation-rmse:6.34116                                                                                           
[938]	validation-rmse:6.34091                                                                                           
[939]	validation-rmse:6.34089                                                                                           
[940]	validation-rmse:6.34033                                                                                           
[941]	validation-rmse:6.34019                                                                                           
[942]	validation-rmse:6.34014                                                                                           
[943]	validation-rmse:6.34010                                                                                           
[944]	validation-rmse:6.33992   

It will hell lot of time to get the optimized metric so in the meantime lets check out the runs which have done till now and lets compare

To find all the runs using our xgboost model, 
just like a SQL search for  
```SQL
tags.model = "xgboost"
```
(since we used a tag model for xgboost)
![query_xgboost](./images/query_xgboost.jpg)


Select all the runs and hit compare...

![compare_xgb_run](./images/compare_xgb_run.jpg)

Here we get to look at how w.r.t each parameter our `rmse` is changing  
since we are interested in parameters having minimum `rmse`  
we will go to bottom right of the graph and just click on it and start dragging it upward

like this : https://youtu.be/iaJz-T7VWec?t=735 to visually have a look how parameters are affecting our `rmse`

Similarily to find the relation of each parameter w.r.t `rmse` using scatter plot and contour plot have a look at this [video](https://youtu.be/iaJz-T7VWec?t=793)


## AutoLogging 

Instead of manually logging each model parameters and metrics, we initialize `autolog()` method to automatically track the parameters 

This only works for certain frameworks about which you can find [here](https://www.mlflow.org/docs/latest/tracking.html#automatic-logging) 

In [39]:
best_params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:linear',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=500,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

# please scroll down for next part as output is just toooo long ... :(

2022/06/14 16:14:56 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c7d9bd1ed20542888bf9caf2faaa0348', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[16:14:57] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	validation-rmse:6.86273
[28]	validation-rmse:6.82111
[29]	validation-rmse:6.78450
[30]	valida

[273]	validation-rmse:6.38673
[274]	validation-rmse:6.38660
[275]	validation-rmse:6.38649
[276]	validation-rmse:6.38615
[277]	validation-rmse:6.38595
[278]	validation-rmse:6.38585
[279]	validation-rmse:6.38558
[280]	validation-rmse:6.38529
[281]	validation-rmse:6.38510
[282]	validation-rmse:6.38480
[283]	validation-rmse:6.38455
[284]	validation-rmse:6.38429
[285]	validation-rmse:6.38410
[286]	validation-rmse:6.38377
[287]	validation-rmse:6.38341
[288]	validation-rmse:6.38321
[289]	validation-rmse:6.38288
[290]	validation-rmse:6.38241
[291]	validation-rmse:6.38218
[292]	validation-rmse:6.38187
[293]	validation-rmse:6.38155
[294]	validation-rmse:6.38099
[295]	validation-rmse:6.38073
[296]	validation-rmse:6.38059
[297]	validation-rmse:6.38034
[298]	validation-rmse:6.38019
[299]	validation-rmse:6.38001
[300]	validation-rmse:6.37986
[301]	validation-rmse:6.37969
[302]	validation-rmse:6.37924
[303]	validation-rmse:6.37890
[304]	validation-rmse:6.37895
[305]	validation-rmse:6.37869
[306]	vali

2022/06/14 16:16:17 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/06/14 16:16:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pathikx/miniconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


If you click on this particular run in MLFlow UI then in the `Artifacts` lies the real beauty....

It stores literally everything which can be useful for us

1. Dependencies 

![dependencies](./images/dependencies.jpg)

2. Python Environment info 

![env_info](./images/env_info.jpg)

3. Feature Importances

4. Metrics 

![metrics](./images/metrics.jpg)

So that's it !!
![noise](https://c.tenor.com/N66me6-wOwMAAAAM/nice-good.gif)